<b> This code is designed to organize model data for ELI and organize it into the respective bins of ENSO. From there it is capable of creating a relationship of precipitation responses relative to the particular ELI bin. </b>

In [1]:
import numpy as np  
from sklearn.linear_model import LinearRegression
import xarray as xr
from metpy.constants import g, omega as om, Re as a
from metpy.units import units

import metpy.calc as mpcalc
from metpy.units import units
from netCDF4 import num2date
import pandas as pd

In [2]:
# Upload ELI data for HAD - Historical
dir = '/chinook/dallmann/MPI_SST/'
fn_his = 'ELI_MPI_2020-2064_DJFmean.nc'

data_his = xr.open_dataset(dir+fn_his)

# Output directory and file
dro = '/chinook/dallmann/ENSO_bins/'
fn_output = 'MPI_future_nino_strong_DJFmean.nc'

data_his

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 46)
Coordinates:
  * time       (time) datetime64[ns] 2019-12-01 ... 2064-10-16T12:00:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    eli        (time) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.7.2 (http://mpimet.mpg.de/...
    history:      Thu Apr 08 22:37:36 2021: cdo yearmean -selmon,10,11,12 ELI...
    Conventions:  CF-1.4
    frequency:    year
    CDO:          Climate Data Operators version 1.7.2 (http://mpimet.mpg.de/...

In [3]:
# Parse the monthly data for DJF

#eli_had_DJF_his = data_had_his.where(data_had_his['time.season'] == 'DJF',drop = True)

#eli_had_DJF_his['eli'].values

# Seasonally average the data over DJF and change time to year

#eli_djf_had_his = eli_had_DJF_his.rolling(min_periods=3, center=True, time=3).mean()
eli_djf_his = data_his.groupby('time.year').mean('time')

eli_djf_his = eli_djf_his.shift(year=1)
eli_djf_his

<xarray.Dataset>
Dimensions:  (year: 46)
Coordinates:
  * year     (year) int64 2019 2020 2021 2022 2023 ... 2060 2061 2062 2063 2064
Data variables:
    eli      (year) float64 nan 179.3 166.0 173.9 ... 182.9 175.0 174.5 170.6

Organize the data in ENSO bins relative to ELI

In [7]:
Nino_strong = eli_djf_his['eli'].where(eli_djf_his >= 179, drop = True)
Nino_moderate = eli_djf_his.where((eli_djf_his['eli'] >= 170) & (eli_djf_his['eli'] < 180), drop = True) 
Nino_weak = eli_djf_his.where((eli_djf_his['eli'] >= 162) & (eli_djf_his['eli'] < 170), drop = True) 
    
Neutral_ENSO = eli_djf_his.where((eli_djf_his['eli'] >= 158) & (eli_djf_his['eli'] < 162), drop = True) 

Nina_weak = eli_djf_his.where((eli_djf_his['eli'] >= 155) & (eli_djf_his['eli'] < 158), drop = True) 
Nina_moderate = eli_djf_his.where((eli_djf_his['eli'] >= 152) & (eli_djf_his['eli'] < 155), drop = True) 
Nina_strong = eli_djf_his['eli'].where(152 > eli_djf_his, drop = True)



Create a new array and download to directory

In [5]:
Nino_strong.to_netcdf(path=dro+fn_output)